# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

e.g.: Import the dataset with mongoimport --db uk_food --collection establishments --file establishments.json --jsonArray

In [1]:
!pip install pymongo

# Import dependencies
from pymongo import MongoClient
from pprint import pprint


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [3]:
# confirm that our new database was created
print(mongo.list_database_names())


['admin', 'config', 'local', 'uk_food']


In [4]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [5]:
# review the collections in our new database
print(db.list_collection_names())

['establishments']


In [6]:
# review a document in the establishments collection
pprint(db.establishments.find_one())

{'AddressLine1': 'Walmer And Kingsdown Golf Club',
 'AddressLine2': 'The Leas',
 'AddressLine3': 'Kingsdown',
 'AddressLine4': 'Kent',
 'BusinessName': 'The Halfway Hut',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'ChangesByServerID': 0,
 'Distance': 4587.792282474894,
 'FHRSID': 1531040,
 'LocalAuthorityBusinessID': 'PI/000085904',
 'LocalAuthorityCode': '182',
 'LocalAuthorityEmailAddress': 'publicprotection@dover.gov.uk',
 'LocalAuthorityName': 'Dover',
 'LocalAuthorityWebSite': 'http://www.dover.gov.uk/',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT14 8EP',
 'RatingDate': '1901-01-01T00:00:00',
 'RatingKey': 'fhrs_awaitinginspection_en-gb',
 'RatingValue': 'AwaitingInspection',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('66e474458e79aab32bf6627d'),
 'geocode': {'latitude': '51.1787284', 'longitude': '1.4032986'},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/1531040',
            'rel': 'self'}],
 'meta':

In [7]:
# assign the collection to a variable
establishments = db['establishments']

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [22]:
# Create a dictionary for the new restaurant data
penang_flavours = {
    "BusinessName": "Penang Flavours",
    "BusinessType": "Restaurant/Cafe/Canteen",
    "AddressLine1": "123 Greenwich High Road",
    "AddressLine2": "",
    "AddressLine3": "",
    "AddressLine4": "",
    "PostCode": "SE10 8JA",
    "PhoneNumber": "02081234567",
    "LocalAuthorityCode": "Greenwich",
    "LocalAuthorityName": "Greenwich Borough Council",
    "RatingValue": None,  # Not yet rated
    "RatingDate": None,  # Not yet rated
    "Hygiene": None,  # No hygiene score yet
    "Latitude": 51.47879,  # Example coordinates
    "Longitude": -0.010677,  # Example coordinates
    "Geocode": {
        "longitude": -0.010677,
        "latitude": 51.47879
    },
    "BusinessTypeID": 1,  # Example, Restaurant type
    "NewBusiness": True,  # Mark as new
    "Halal": True  # Mark as a Halal restaurant
}

In [23]:
# Insert the new restaurant into the collection
db.establishments.insert_one(penang_flavours)

InsertOneResult(ObjectId('66e47b31b2917a586d32734b'), acknowledged=True)

In [24]:
# Check that the new restaurant was inserted
db.establishments.find_one({"BusinessName": "Penang Flavours"})

{'_id': ObjectId('66e47b31b2917a586d32734b'),
 'BusinessName': 'Penang Flavours',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'AddressLine1': '123 Greenwich High Road',
 'AddressLine2': '',
 'AddressLine3': '',
 'AddressLine4': '',
 'PostCode': 'SE10 8JA',
 'PhoneNumber': '02081234567',
 'LocalAuthorityCode': 'Greenwich',
 'LocalAuthorityName': 'Greenwich Borough Council',
 'RatingValue': None,
 'RatingDate': None,
 'Hygiene': None,
 'Latitude': 51.47879,
 'Longitude': -0.010677,
 'Geocode': {'longitude': -0.010677, 'latitude': 51.47879},
 'BusinessTypeID': 1,
 'NewBusiness': True,
 'Halal': True}

2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [28]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
result = db.establishments.find_one(
    {"BusinessType": "Restaurant/Cafe/Canteen"},  # Query for BusinessType
    {"BusinessTypeID": 1, "BusinessType": 1, "_id": 0}  # Projection to return only BusinessTypeID and BusinessType
    )

print(result)

{'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': 1}


3. Update the new restaurant with the `BusinessTypeID` you found.

In [29]:
# Update the new restaurant with the correct BusinessTypeID
db.establishments.update_one(
    {"BusinessName": "Penang Flavours"},  # Filter by BusinessName
    {"$set": {"BusinessTypeID": 1}}
)

UpdateResult({'n': 1, 'nModified': 0, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [30]:
# Confirm that the new restaurant was updated
# Confirm that "Penang Flavours" was updated with the correct BusinessTypeID
updated_restaurant = db.establishments.find_one(
    {"BusinessName": "Penang Flavours"},  # Filter by the business name
    {"BusinessName": 1, "BusinessTypeID": 1, "_id": 0}  # Project only BusinessName and BusinessTypeID
)

# Print the result to verify the update
print(updated_restaurant)


{'BusinessName': 'Penang Flavours', 'BusinessTypeID': 1}


4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [31]:
# Find how many documents have LocalAuthorityName as "Dover"
dover_count = db.establishments.count_documents({"LocalAuthorityName": "Dover"})

# Print the result
print(f"Number of establishments in Dover: {dover_count}")


Number of establishments in Dover: 994


In [32]:
# Delete all documents where LocalAuthorityName is "Dover"
db.establishments.delete_many({"LocalAuthorityName": "Dover"})

DeleteResult({'n': 994, 'ok': 1.0}, acknowledged=True)

In [34]:
# Check if any remaining documents include Dover
dover_count_after = db.establishments.count_documents({"LocalAuthorityName": "Dover"})
# Print the result to confirm deletion
print(f"Number of establishments in Dover after deletion: {dover_count_after}")

Number of establishments in Dover after deletion: 0


In [37]:
# Check that other documents remain with 'find_one'
remaining_document = db.establishments.find_one()

# Print the result
print(remaining_document)


{'_id': ObjectId('66e474458e79aab32bf66563'), 'FHRSID': 1043695, 'ChangesByServerID': 0, 'LocalAuthorityBusinessID': 'PI/000073616', 'BusinessName': 'The Pavilion', 'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': 1, 'AddressLine1': 'East Cliff Pavilion', 'AddressLine2': 'Wear Bay Road', 'AddressLine3': 'Folkestone', 'AddressLine4': 'Kent', 'PostCode': 'CT19 6BL', 'Phone': '', 'RatingValue': '5', 'RatingKey': 'fhrs_5_en-gb', 'RatingDate': '2018-04-04T00:00:00', 'LocalAuthorityCode': '188', 'LocalAuthorityName': 'Folkestone and Hythe', 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk', 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk', 'scores': {'Hygiene': 5, 'Structural': 5, 'ConfidenceInManagement': 5}, 'SchemeType': 'FHRS', 'geocode': {'longitude': '1.195625', 'latitude': '51.083812'}, 'RightToReply': '', 'Distance': 4591.765489457773, 'NewRatingPending': False, 'meta': {'dataSource': None, 'extractDate': '0001-01-01T00:00:00', 'itemCount': 0, 

5. Some of the number values are stored as strings, when they should be stored as numbers.

Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [38]:
# Change the data type from String to Decimal for longitude and latitude
db.establishments.update_many(
    {},  # Apply to all documents
    [
        {"$set": {
            "longitude": {"$toDouble": "$longitude"},
            "latitude": {"$toDouble": "$latitude"}
        }}
    ]
)

UpdateResult({'n': 38786, 'nModified': 38786, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

Use `update_many` to convert `RatingValue` to integer numbers.

In [19]:
# Set non 1-5 Rating Values to Null
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
establishments.update_many({"RatingValue": {"$in": non_ratings}}, [ {'$set':{ "RatingValue" : None}} ])

UpdateResult({'n': 4138, 'nModified': 4138, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [39]:
# Change the data type from String to Integer for RatingValue
db.establishments.update_many(
    {"RatingValue": {"$nin": non_ratings, "$ne": None}},  # Only apply to valid numeric RatingValues
    [
        {"$set": {"RatingValue": {"$toInt": "$RatingValue"}}}  # Convert to integer
    ]
)

UpdateResult({'n': 34694, 'nModified': 34694, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [40]:
# Check that the coordinates and rating value are now numbers
sample_document = db.establishments.find_one(
    {},  # Find any document
    {"BusinessName": 1, "latitude": 1, "longitude": 1, "RatingValue": 1, "_id": 0}  # Project only relevant fields
)

# Print the result to verify
print(sample_document)


{'BusinessName': 'The Pavilion', 'RatingValue': 5, 'longitude': None, 'latitude': None}
